In [2]:
import numpy as np
import pycosat
import time

In [3]:
# this cell is responsible for generating valid filled circledokus

# for a circledoku of 3 rings we represent in an array as follows
#
# ring:
# [[1 1 1 1 1 1]
#  [0 0 0 0 0 0]
#  [0 0 0 0 0 0]]
#
# slice:
# [[1 0 0 1 0 0]
#  [1 0 0 1 0 0]
#  [1 0 0 1 0 0]]
#
# wedge:
# [[1 1 0 0 0 0]
#  [1 1 0 0 0 0]
#  [1 1 0 0 0 0]]

def gen_full_circledoku(n): # a circledoku with n-rings
    # this function fills a circledoku with a valid configuration
    # it is later filtered to become a unique proper puzzle with minimal givens, outside of this cell
    succes = False
    n_attempt = 0
    while not succes: 
        # if the computer runs into a dead-end by random assignment, just stop and start over
        # this is very inefficient especially for larger circledoku's
        # but in this generation step we are not troubled by efficiency
        succes, doku = attempt_full_circledoku(n)
#         past_attempt.append(doku)
        n_attempt += 1
    print("succes took ", n_attempt, " attempts")
    return doku
            
def attempt_full_circledoku(n):
    maxDigit = 2*n
    doku = np.zeros((n, maxDigit), dtype=np.int)
    for i in range(n):
        values = [] # a list of numbers to choose from
        for n in range(1, maxDigit+1):
            values.append(n)
        for j in range(maxDigit):
            value = values[np.random.randint(0, len(values))]             
            gen = [i, j, value]
            count = 1
            while not is_gen_valid(doku, gen):
                idx = np.random.randint(0, len(values))
                gen = [i, j, values[idx]]
                values.pop(idx)
                count += 1
                if len(values) == 0:
                    return False, doku
            doku[i, j] = gen[2]
    return True, doku

def is_gen_valid(doku, gen):
    # variable 'gen' is the digit the algorithm wants to genereate at certain position
    # its syntax is [x, y, digit]
    if is_gen_valid_for_ring(doku, gen) and is_gen_valid_for_slice(doku, gen) and is_gen_valid_for_wedge(doku, gen):
        return True
    else:
        return False
    
def is_gen_valid_for_ring(doku, gen):
    ring_content = doku[gen[0]][:]
    if gen[2] in ring_content:
        return False
    else:
        return True

def is_gen_valid_for_slice(doku, gen):
    n = len(doku)
    n_slice = gen[1] % n
    slice_content = doku.transpose()[n_slice::n][:]
    if gen[2] in slice_content:
        return False
    else:
        return True
    
def is_gen_valid_for_wedge(doku, gen):
    n = len(doku)
    n_wedge = int(gen[1] / 2)
    wedge_content = doku.transpose()[n_wedge*2:n_wedge*2+2][:]
    if gen[2] in wedge_content:
        return False
    else:
        return True

def construct_database():
    for n in [4,6,7,10]:
        k = 50 # amount of puzzles to generate per size/nRings
        for i in range(k):
            tic = time.clock()
            doku = gen_full_circledoku(n)
            toc = time.clock()
            print("Circledoku order:", n, ", nr: ", i, " took ", toc-tic, "seconds")
            filename_small = "CircleDoku_nrings" + str(n) + "_#" +str(i)
            np.savetxt(filename_small, doku, fmt='%i', delimiter=',', newline='\n' )        
        collect_smalls_to_big(n, k-1)
        
def collect_smalls_to_big(nRings, nPuzzles):
    filename_big = "CircleDokuDatabase_for_size_" + str(nRings)
    with open(filename_big, 'w'): pass # clear the file
    f=open(filename_big,'ab')
    for i in range(nPuzzles):
        filename_small = "CircleDoku_nrings"+str(nRings)+"_#"+str(nPuzzles)
        doku = np.loadtxt(filename_small, delimiter=',')
        np.savetxt(f, doku, fmt='%i', delimiter=',', newline='\n' )
    f.close()

In [28]:
# in this cell you can see how generating database works

# the following call will create 50 circledokus for a few sizes (indicated in the function)
# it writes every sudoku to its own txt file
# when all is done it writes all to a bigger file that contains all puzzles of size nRings
# the advantage is that heavy calculation is only performed when the big txt file is closed
# when the calculation is broken off before 50, all the intermediate puzzles will still be saved
# in their respective small_txt file, and can be read out with a separate call to collect_smalls_to_big
# However, this might take a while
construct_database() 
#collect_smalls_to_big(3, 8) 

KeyboardInterrupt: 

In [4]:
# this file read the big txt file to a 3d array, puzzles-rows-columns

def load_circleDoku_database(n):
    filename = "CircleDokuDatabase_for_size_"+str(n)
    a = np.loadtxt(filename, delimiter=',' )
    nDoku = int(len(a)/n)
    a = a.reshape(((nDoku, n, 2*n)))
    return a

load_circleDoku_database(5)

array([[[  8.,   4.,   3.,   2.,  10.,   6.,   1.,   9.,   5.,   7.],
        [  3.,   9.,   1.,   4.,   5.,   7.,  10.,   6.,   8.,   2.],
        [  5.,   2.,  10.,   9.,   1.,   4.,   8.,   7.,   3.,   6.],
        [  1.,   6.,   8.,   7.,   3.,   2.,   5.,   4.,  10.,   9.],
        [ 10.,   7.,   5.,   6.,   8.,   9.,   3.,   2.,   1.,   4.]],

       [[  5.,   2.,   8.,   6.,   9.,   3.,   7.,   4.,  10.,   1.],
        [  7.,   4.,   9.,   3.,   5.,   1.,  10.,   6.,   8.,   2.],
        [  8.,   6.,   7.,   1.,  10.,   4.,   9.,   2.,   5.,   3.],
        [  9.,   3.,  10.,   4.,   8.,   2.,   5.,   1.,   7.,   6.],
        [ 10.,   1.,   5.,   2.,   7.,   6.,   8.,   3.,   9.,   4.]],

       [[  8.,   7.,   9.,  10.,   5.,   1.,   3.,   4.,   6.,   2.],
        [  5.,  10.,   3.,   2.,   6.,   4.,   9.,   1.,   8.,   7.],
        [  6.,   4.,   5.,   1.,   3.,   7.,   8.,   2.,   9.,  10.],
        [  9.,   2.,   6.,   4.,   8.,  10.,   5.,   7.,   3.,   1.],
        [  3.,  